## 1. Importing Libraries

In [1]:
import os, glob, shutil
import numpy as np 
import torch
import torchvision
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision import transforms
from torch.nn import functional as f
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torch import optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torchinfo import summary
from torchvision.models import vgg16

## 2. Loading Data

In [3]:
train_dir = "./dataset/train"
test_dir = "./dataset/test"

train_cat_dir = os.path.join(train_dir, "cats")
train_dog_dir =  os.path.join(train_dir, "dogs")

def create_dir():
    try:
        os.makedirs(train_cat_dir, exist_ok= True)
        os.makedirs(train_dog_dir, exist_ok= True)

    except:
        pass

def move_to_respectice_directories(
    source_folder,
    des_folder,
    keyword    
    ):
    for image_file_path in glob.glob(os.path.join(source_folder, "**")):
        try:
            image_file_name = os.path.basename(image_file_path)
            if keyword in image_file_name:
                shutil.move(
                    os.path.join(source_folder, image_file_name),
                    os.path.join(des_folder, image_file_name)
                )
        
        except:
            pass
    
create_dir()

move_to_respectice_directories(train_dir, train_cat_dir, 'cat')
move_to_respectice_directories(train_dir, train_dog_dir, "dog")
        

In [ ]:
dataset = ImageFolder(train_dir)

train_data, val_data, train_label, val_label = train_test_split(
    dataset.imgs, dataset.targets,
    test_size = 0.1, random_state= 42
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"The devie use for computation{device}")

img_size = 256

class DogVSCatDataset(Dataset):
    def __init__(self, dataset, transform= None, img_size = 256):
        self.dataset = self.is_channel_RGB(dataset)
        self.transform = transform

    def __len__(self):
        ''' 
        get the length of the dataset
        '''
        return len(self.dataset)
    
    def __getitem__(self, idx) -> Any:
        """
        get the images and labels, apply the transformation / augmentation
        """
        image = Image.open(self.dataset[idx][0])
        label = self.dataset[idx][1]
        if self.transform:
            image = self.transform(image)
        
        return image, label
    
    def collate_fn(self, batch):
        """
        preprocess images, laels before loading to training/validation
        """
        imgs, labels = [], []
        for im, label in batch:
            im = self.

    def is_channel_RGB(self,dataset):
        ''' 
        Include only the images that has RGB channel in it. Exclude the rest
        '''
        dataset_rgb = []
        for idx in range(len(dataset)):
            if Image.open(dataset['idx'][0]).getbands() == ("R", "G", "B"):

                dataset_rgb.append(dataset[idx])

        return dataset_rgb
    
# train transform
train_transform = transforms.Compose([
    transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# val transform
val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

train_dataset = DogVSCatDataset(train_data, transform= train_transform, img_size= img_size)
val_dataset = DogVSCatDataset(val_data, transform= val_transform, img_size= img_size)

train_dataloader = DataLoader(train_dataset, batch_size= 16, shuffle= True)
val_dataloader = DataLoader(val_dataset, batch_size= 2, shuffle= True)

